In [170]:
import glob
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yaml

sns.set_theme()
sns.set_context("paper")
sns.set(font_scale=1.5)

ibm_colors = [
    "#648FFF",
    # "#785EF0",
    "#DC267F",
    # "#FE6100",
    "#FFB000",
]

In [171]:
fig_dir = Path.home() / "dev/data/exp/RHCR/figures"
fig_dir.mkdir(parents=True, exist_ok=True)

RHCR_dir = Path.home() / "dev/data/exp/RHCR"

exp_runs = []
# PBS
exp_runs += glob.glob("2024-12-11*/", root_dir=RHCR_dir)
# LaCAM
exp_runs += glob.glob("2024-12-13*/", root_dir=RHCR_dir)
# Sequential LaCAM
# exp_runs += glob.glob("exp_sequential/2024-12-16*/", root_dir=RHCR_dir)

data_dirs = [RHCR_dir / run for run in exp_runs]

In [172]:
def read_yaml(file_path):
    with open(file_path, mode="r") as file:
        data = yaml.safe_load(file)
    return data


configs = [read_yaml(data_dir / "config.yaml") for data_dir in data_dirs]

df = pd.concat(
    [pd.read_csv(data_dir / "result.csv") for data_dir in data_dirs], ignore_index=True
)

df["solver"] = df["solver"].replace("LaCAM", "pLaCAM")
df["solver"] = df["solver"].replace("LaCAMSequential", "Vanilla LaCAM")

# map solver to color
ibm_palette = dict(zip(["Vanilla LaCAM", "PBS", "pLaCAM"], ibm_colors))

In [ ]:
maps = df["map"].unique()
x_vars = [
    "agentNum",
    # "simulation_window",
]
xlabels = {
    "agentNum": "Agent Quantity",
    "simulation_window": "Simulation Window Size (steps)",
}
for map_name in maps:
    for i, x_var in enumerate(x_vars):
        y_vars = [
            "completed_tasks",
            "runtime_s",
            "success_rate",
        ]
        ylabels = {
            "completed_tasks": "Completed Tasks $\\rightarrow$",
            "runtime_s": "$\\leftarrow$ Runtime (s)",
            "success_rate": "Success Rate (%) $\\rightarrow$",
        }
        for j, y_var in enumerate(y_vars):
            fig = plt.figure()
            ax = plt.gca()
            filtered: pd.DataFrame = df[df["map"] == map_name]

            if y_var == "success_rate":
                success_rate_df = (
                    filtered.groupby(["solver", x_var])
                    .apply(
                        lambda group: group["completed_tasks"].notna().sum()
                        / len(group)
                        * 100,
                        include_groups=False,
                    )
                    .reset_index(name="success_rate")
                )

                success_rate_df[x_var] = success_rate_df[x_var].astype(str)

                melted = success_rate_df.melt(
                    id_vars=["solver", x_var],
                    var_name="measurement",
                    value_name="value",
                )
                melted["solver"] = pd.Categorical(
                    melted["solver"], melted["solver"].unique().sort()
                )

                ax = sns.lineplot(
                    data=melted,
                    x=x_var,
                    y="value",
                    hue="solver",
                    marker="o",
                    ax=ax,
                    legend=True,
                    zorder=1,
                    palette=ibm_palette,
                )
            else:
                melted = filtered.melt(
                    id_vars=["solver", x_var],
                    var_name="measurement",
                    value_vars=[y_var],
                )
                # consistent order
                melted["solver"] = pd.Categorical(
                    melted["solver"], melted["solver"].unique().sort()
                )

                ax = sns.stripplot(
                    data=melted,
                    x=x_var,
                    y="value",
                    hue="solver",
                    dodge=True,
                    alpha=0.1,
                    jitter=0.1,
                    ax=ax,
                    legend=False,
                    zorder=1,
                    palette=ibm_palette,
                )

                ax = sns.pointplot(
                    data=melted,
                    x=x_var,
                    y="value",
                    hue="solver",
                    dodge=0.8 - 0.8 / len(melted["solver"].unique()),
                    errorbar=None,
                    markers="x",
                    linestyle="none",
                    ax=ax,
                    legend=True,
                    estimator="median",
                    palette=ibm_palette,
                )

            if y_var == "success_rate":
                ax.set_ylim(-5, 105)

            ax.set_xlabel(xlabels[x_var])
            ax.set_ylabel(ylabels[y_var])
            ax.grid(True)

            if (legend := ax.get_legend()) is not None:
                legend.remove()
            if j == len(y_vars) - 1:
                legend_handles, legend_labels = ax.get_legend_handles_labels()
                for handle in legend_handles:
                    handle.set_linewidth(0)
                    handle.set_marker("o")
                    handle.set_markersize(20)
                fig_legend = fig.legend(
                    legend_handles,
                    legend_labels,
                    loc="lower center",
                    bbox_to_anchor=(0.5, -0.3),
                    ncol=len(melted["solver"].unique()),
                    frameon=True,
                    title="Solver",
                )

            ax.tick_params(axis="x", rotation=90)

            fig.savefig(fig_dir / f"{Path(map_name).stem}_{x_var}_{y_var}.pdf", bbox_inches="tight")